In [1]:
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/data/data_prep/')

In [5]:
files = []
all_symbols = set()

In [6]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [7]:
def calculate_ma(df, ema=True, sma=True, calc_windows=[]):
    # Function to apply moving averages
    def apply_moving_averages(group):
        for window in tqdm(calc_windows):
            for price in ['open', 'high', 'low', 'close', 'volume']:
                if ema:
                    group[f'{price}_ema_{window}m'] = group[price].ewm(span=window, adjust=False).mean()
                if sma:
                    group[f'{price}_sma_{window}m'] = group[price].rolling(window=window).mean()
        return group

    # Apply function by group
    return df.groupby('symbol').apply(apply_moving_averages)

In [8]:
for path in paths:
    name = path.split('/')[-1].split('.')[0].split('_')[-1]
    if 'min' in name:
        window = int(name.replace('min', ''))
        period = 'min'
    elif 'D' in name:
        window = int(name.replace('D', ''))
        period = 'D'
    if period=='D':
        if window < 100:
            calc_windows = [2, 3, 5, 8, 10, 12, 15, 20, 26, 50, 100, 200]
        else: 
            calc_windows = [5, 8, 10, 12, 20, 26, 50]
    else:
        if window <= 30:
            calc_windows = [2, 3, 5, 8, 12, 10, 20, 26]
        else:
            calc_windows = [5, 8, 10, 12, 20, 26, 50, 100, 200]
    print(path)
    df = pd.read_parquet(path)
    df = calculate_ma(df, calc_windows=calc_windows).droplevel(0)
    df.to_parquet(path.replace('data_prep', 'feature_prep'))
    print(f"written to {path.replace('data_prep', 'feature_prep')}")    

s3://sisyphus-general-bucket/AthenaInsights/data/data_prep/stock_bars_100D.parquet


100%|██████████| 7/7 [00:00<00:00, 114.39it/s]


                         open     high      low    close     volume symbol  \
us_eastern_timestamp                                                         
2020-01-02            28.5375  28.6631  17.0000  22.1176  1009552.0    DAX   
2020-04-11            22.4500  27.9883  20.7364  27.8300  1353828.0    DAX   
2020-07-20            27.9200  29.7200  26.5624  26.6800  1243774.0    DAX   
2020-10-28            25.6400  32.0299  25.1275  31.0250   975409.0    DAX   
2021-02-05            31.1600  34.6100  30.6000  34.5538   699338.0    DAX   

                      open_ema_5m  open_sma_5m  high_ema_5m  high_sma_5m  ...  \
us_eastern_timestamp                                                      ...   
2020-01-02              28.537500          NaN    28.663100          NaN  ...   
2020-04-11              26.508333          NaN    28.438167          NaN  ...   
2020-07-20              26.978889          NaN    28.865444          NaN  ...   
2020-10-28              26.532593          NaN  

100%|██████████| 12/12 [00:00<00:00, 92.98it/s]


                         open     high     low  close   volume symbol  \
us_eastern_timestamp                                                    
2020-01-02            28.5375  28.6631  27.800  28.50  14355.0    DAX   
2020-01-12            28.2400  28.5700  28.225  28.48  44010.0    DAX   
2020-01-22            28.4000  28.6499  27.100  27.10  57643.0    DAX   
2020-02-01            27.2300  28.3900  27.230  27.95  20109.0    DAX   
2020-02-11            28.1600  28.5599  27.940  27.94  18150.0    DAX   

                      open_ema_2m  open_sma_2m  high_ema_2m  high_sma_2m  ...  \
us_eastern_timestamp                                                      ...   
2020-01-02              28.537500          NaN    28.663100          NaN  ...   
2020-01-12              28.339167     28.38875    28.601033     28.61655  ...   
2020-01-22              28.379722     28.32000    28.633611     28.60995  ...   
2020-02-01              27.613241     27.81500    28.471204     28.51995  ...   
20

...

                         open     high      low    close   volume symbol  \
us_eastern_timestamp                                                       
2020-01-02            28.5375  28.6631  27.9701  27.9900   4444.0    DAX   
2020-01-05            27.8000  28.0658  27.8000  28.0400   1640.0    DAX   
2020-01-08            28.1550  28.5172  28.1550  28.5000   8271.0    DAX   
2020-01-11            28.2400  28.5300  28.2250  28.5300   3965.0    DAX   
2020-01-14            28.3419  28.5000  28.3158  28.3801  27518.0    DAX   

                      open_ema_2m  open_sma_2m  high_ema_2m  high_sma_2m  ...  \
us_eastern_timestamp                                                      ...   
2020-01-02              28.537500          NaN    28.663100          NaN  ...   
2020-01-05              28.045833     28.16875    28.264900     28.36445  ...   
2020-01-08              28.118611     27.97750    28.433100     28.29150  ...   
2020-01-11              28.199537     28.19750    28.497700   

100%|██████████| 12/12 [00:00<00:00, 185.44it/s]


                         open     high      low    close    volume symbol  \
us_eastern_timestamp                                                        
2020-01-02            28.5375  28.6631  27.1000  27.9400  154267.0    DAX   
2020-02-21            27.9418  27.9418  17.0000  22.1176  855285.0    DAX   
2020-04-11            22.4500  24.9600  20.7364  24.5780  449912.0    DAX   
2020-05-31            24.8600  27.9883  24.8350  27.8300  903916.0    DAX   
2020-07-20            27.9200  29.7200  26.9500  28.9399  910442.0    DAX   

                      open_ema_2m  open_sma_2m  high_ema_2m  high_sma_2m  ...  \
us_eastern_timestamp                                                      ...   
2020-01-02              28.537500          NaN    28.663100          NaN  ...   
2020-02-21              28.140367     28.23965    28.182233     28.30245  ...   
2020-04-11              24.346789     25.19590    26.034078     26.45090  ...   
2020-05-31              24.688930     23.65500    27.33

100%|██████████| 12/12 [00:00<00:00, 153.30it/s]


                         open     high     low    close   volume symbol  \
us_eastern_timestamp                                                      
2020-01-02            28.5375  28.6631  27.800  27.8948   5419.0    DAX   
2020-01-07            28.0400  28.5172  28.040  28.5000   8936.0    DAX   
2020-01-12            28.2400  28.5300  28.225  28.3801  31483.0    DAX   
2020-01-17            28.5600  28.5700  28.320  28.4800  12527.0    DAX   
2020-01-22            28.4000  28.6499  28.035  28.2700  16493.0    DAX   

                      open_ema_2m  open_sma_2m  high_ema_2m  high_sma_2m  ...  \
us_eastern_timestamp                                                      ...   
2020-01-02              28.537500          NaN    28.663100          NaN  ...   
2020-01-07              28.205833     28.28875    28.565833     28.59015  ...   
2020-01-12              28.228611     28.14000    28.541944     28.52360  ...   
2020-01-17              28.449537     28.40000    28.560648     28.55